# Pandas Data Structures

In [36]:
# Creating DataFrame objects from files, API requests, SQL queries, and other Python objects
# Inspecting DataFrame objects and calculating summary statistics
# Grabbing subsets of the data via selection, slicing, indexing, and filtering
# Adding and removing data

In [37]:
# files:  https://github.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/tree/master/ch_02
# US Geological Survey (USGS) 

In [38]:
# earthquakes.csv pulled from September 18, 2018 through October 13, 2018
# example_data.csv contains five rows and a subset of the columns from the earthquakes.csv
# tsunamis.csv file is a subset of the data in the earthquakes.csv
# quakes.db file contains a SQLite database with a single table for the tsunamis data

In [39]:
# NumPy provides the n-dimensional arrays that pandas builds upon - implemented as Python classes

In [40]:
import numpy as np

data = np.genfromtxt(
    'data/example_data.csv', delimiter=';', 
    names=True, dtype=None, encoding='UTF'
)
data

array([('2018-10-13 11:10:23.560', '262km NW of Ozernovskiy, Russia', 'mww', 6.7, 'green', 1),
       ('2018-10-13 04:34:15.580', '25km E of Bitung, Indonesia', 'mww', 5.2, 'green', 0),
       ('2018-10-13 00:13:46.220', '42km WNW of Sola, Vanuatu', 'mww', 5.7, 'green', 0),
       ('2018-10-12 21:09:49.240', '13km E of Nueva Concepcion, Guatemala', 'mww', 5.7, 'green', 0),
       ('2018-10-12 02:52:03.620', '128km SE of Kimbe, Papua New Guinea', 'mww', 5.6, 'green', 1)],
      dtype=[('time', '<U23'), ('place', '<U37'), ('magType', '<U3'), ('mag', '<f8'), ('alert', '<U5'), ('tsunami', '<i4')])

In [41]:
#  information about the dimensions of the array and the data types 
data.shape

(5,)

In [42]:
data.dtype

dtype([('time', '<U23'), ('place', '<U37'), ('magType', '<U3'), ('mag', '<f8'), ('alert', '<U5'), ('tsunami', '<i4')])

In [45]:
#  NumPy arrays contain a single data type

In [44]:
# select the third index of each row
# to see how long this implementation takes:
%%timeit
max([row[3] for row in data])

5.53 µs ± 244 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [46]:
# dictionary comprehension: keys are the column names and the values are NumPy arrays of the data
# early twice as fast as the list comprehension implementation,
array_dict = {
    col: np.array([row[i] for row in data])
    for i, col in enumerate(data.dtype.names)
}
array_dict

{'time': array(['2018-10-13 11:10:23.560', '2018-10-13 04:34:15.580',
        '2018-10-13 00:13:46.220', '2018-10-12 21:09:49.240',
        '2018-10-12 02:52:03.620'], dtype='<U23'),
 'place': array(['262km NW of Ozernovskiy, Russia', '25km E of Bitung, Indonesia',
        '42km WNW of Sola, Vanuatu',
        '13km E of Nueva Concepcion, Guatemala',
        '128km SE of Kimbe, Papua New Guinea'], dtype='<U37'),
 'magType': array(['mww', 'mww', 'mww', 'mww', 'mww'], dtype='<U3'),
 'mag': array([6.7, 5.2, 5.7, 5.7, 5.6]),
 'alert': array(['green', 'green', 'green', 'green', 'green'], dtype='<U5'),
 'tsunami': array([1, 0, 0, 0, 1])}

In [47]:
%%timeit
array_dict['mag'].max()

2.74 µs ± 138 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [48]:
#a NumPy array of strings
np.array([
    value[array_dict['mag'].argmax()] 
    for key, value in array_dict.items()
])

array(['2018-10-13 11:10:23.560', '262km NW of Ozernovskiy, Russia',
       'mww', '6.7', 'green', '1'], dtype='<U31')

## Series data structure

In [49]:
import pandas as pd

place = pd.Series(array_dict['place'], name='place')
place
# data type of place is object

0          262km NW of Ozernovskiy, Russia
1              25km E of Bitung, Indonesia
2                42km WNW of Sola, Vanuatu
3    13km E of Nueva Concepcion, Guatemala
4      128km SE of Kimbe, Papua New Guinea
Name: place, dtype: object

In [50]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html

### Index class

In [51]:
# row labels enable selection by row
place_index = place.index
place_index

RangeIndex(start=0, stop=5, step=1)

In [54]:
# Index object is built on top of a NumPy array
place_index.values

array([0, 1, 2, 3, 4], dtype=int64)

In [56]:
# arithmetic operations - element-wise
np.array([1, 1, 1]) + np.array([-1, 0, 1])

array([0, 1, 2])

### DataFrame class

In [58]:
# builds upon the Series class and can have many columns, each with its own data type
df = pd.DataFrame(array_dict)
df

,time,place,magType,mag,alert,tsunami
0,2018-10-13 11:10:23.560,"262km NW of Ozernovskiy, Russia",mww,6.7,green,1
1,2018-10-13 04:34:15.580,"25km E of Bitung, Indonesia",mww,5.2,green,0
2,2018-10-13 00:13:46.220,"42km WNW of Sola, Vanuatu",mww,5.7,green,0
3,2018-10-12 21:09:49.240,"13km E of Nueva Concepcion, Guatemala",mww,5.7,green,0
4,2018-10-12 02:52:03.620,"128km SE of Kimbe, Papua New Guinea",mww,5.6,green,1


In [59]:
df.dtypes

time        object
place       object
magType     object
mag        float64
alert       object
tsunami      int32
dtype: object

In [60]:
df.values

array([['2018-10-13 11:10:23.560', '262km NW of Ozernovskiy, Russia',
        'mww', 6.7, 'green', 1],
       ['2018-10-13 04:34:15.580', '25km E of Bitung, Indonesia', 'mww',
        5.2, 'green', 0],
       ['2018-10-13 00:13:46.220', '42km WNW of Sola, Vanuatu', 'mww',
        5.7, 'green', 0],
       ['2018-10-12 21:09:49.240',
        '13km E of Nueva Concepcion, Guatemala', 'mww', 5.7, 'green', 0],
       ['2018-10-12 02:52:03.620', '128km SE of Kimbe, Papua New Guinea',
        'mww', 5.6, 'green', 1]], dtype=object)

In [61]:
df.columns

Index(['time', 'place', 'magType', 'mag', 'alert', 'tsunami'], dtype='object')

In [62]:
# Pandas will only perform the operation when both the index and column match
df + df

,time,place,magType,mag,alert,tsunami
0,2018-10-13 11:10:23.5602018-10-13 11:10:23.560,"262km NW of Ozernovskiy, Russia262km NW of Oze...",mwwmww,13.4,greengreen,2
1,2018-10-13 04:34:15.5802018-10-13 04:34:15.580,"25km E of Bitung, Indonesia25km E of Bitung, I...",mwwmww,10.4,greengreen,0
2,2018-10-13 00:13:46.2202018-10-13 00:13:46.220,"42km WNW of Sola, Vanuatu42km WNW of Sola, Van...",mwwmww,11.4,greengreen,0
3,2018-10-12 21:09:49.2402018-10-12 21:09:49.240,"13km E of Nueva Concepcion, Guatemala13km E of...",mwwmww,11.4,greengreen,0
4,2018-10-12 02:52:03.6202018-10-12 02:52:03.620,"128km SE of Kimbe, Papua New Guinea128km SE of...",mwwmww,11.2,greengreen,2


## Creating a pandas DataFrame

In [63]:
import datetime as dt
import numpy as np
import pandas as pd

In [64]:
# how to make a Series object (series of five random numbers between 0 and 1)
# seed gives a starting point for the generation of pseudorandom numbers

np.random.seed(0) # set a seed for reproducibility
pd.Series(np.random.rand(5), name='random')

0    0.548814
1    0.715189
2    0.602763
3    0.544883
4    0.423655
Name: random, dtype: float64

In [65]:
# Creating a DataFrame object from a Series object
pd.Series(np.linspace(0, 10, num=5)).to_frame()

,0
0,0.0
1,2.5
2,5.0
3,7.5
4,10.0


In [66]:
# Creating a DataFrame from Python Data Structures
# to turn a single Series object into a DataFrame object: use its to_frame() method
# columns can all be different data types
# package the columns in a dictionary

np.random.seed(0) 
pd.DataFrame(
    {
        'random': np.random.rand(5),
        'text': ['hot', 'warm', 'cool', 'cold', None],
        'truth': [np.random.choice([True, False]) for _ in range(5)]
    }, 
    index=pd.date_range(
        end=dt.date(2019, 4, 21),
        freq='1D',
        periods=5, 
        name='date'
    )
)

# By convention, we use _ to hold variables in a loop that we don't care about
# https://hackernoon.com/understanding-the-underscore-of-python-309d1a029edc

,random,text,truth
date,,,
2019-04-17,0.548814,hot,False
2019-04-18,0.715189,warm,True
2019-04-19,0.602763,cool,True
2019-04-20,0.544883,cold,False
2019-04-21,0.423655,None,True


In [67]:
pd.DataFrame([
    {'mag': 5.2, 'place': 'California'},
    {'mag': 1.2, 'place': 'Alaska'},
    {'mag': 0.2, 'place': 'California'},
])

,mag,place
0,5.2,California
1,1.2,Alaska
2,0.2,California


In [68]:
list_of_tuples = [(n, n**2, n**3) for n in range(5)]
list_of_tuples

[(0, 0, 0), (1, 1, 1), (2, 4, 8), (3, 9, 27), (4, 16, 64)]

In [69]:
pd.DataFrame(
    list_of_tuples, 
    columns=['n', 'n_squared', 'n_cubed']
)

,n,n_squared,n_cubed
0,0,0,0
1,1,1,1
2,2,4,8
3,3,9,27
4,4,16,64


In [70]:
pd.DataFrame(
    np.array([
        [0, 0, 0],
        [1, 1, 1],
        [2, 4, 8],
        [3, 9, 27],
        [4, 16, 64]
    ]), columns=['n', 'n_squared', 'n_cubed']
)

,n,n_squared,n_cubed
0,0,0,0
1,1,1,1
2,2,4,8
3,3,9,27
4,4,16,64


## Creating a DataFrame object from the contents of a CSV File

In [88]:
# check how big the file is: Number of lines (row count)

!find /c /v "" data\earthquakes.csv


---------- DATA\EARTHQUAKES.CSV: 9333


In [81]:
# file size
!dir data | findstr "earthquakes.csv"

04.04.22  18:16         3.524.987 earthquakes.csv


In [83]:
files = !dir data
[file for file in files if 'earthquake' in file]

['04.04.22  18:16         3.524.987 earthquakes.csv']

In [84]:
# Examining a few rows

n = 2
with open('data/earthquakes.csv', 'r') as file:
    for _ in range(n):
        print(file.readline())

alert,cdi,code,detail,dmin,felt,gap,ids,mag,magType,mmi,net,nst,place,rms,sig,sources,status,time,title,tsunami,type,types,tz,updated,url

,,37389218,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci37389218&format=geojson,0.008693,,85.0,",ci37389218,",1.35,ml,,ci,26.0,"9km NE of Aguanga, CA",0.19,28,",ci,",automatic,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475395144,https://earthquake.usgs.gov/earthquakes/eventpage/ci37389218



In [89]:
# Column count

import os

with open('data/earthquakes.csv', 'rb') as file:
    file.seek(-2, os.SEEK_END)
    while file.read(1) != b'\n':
        file.seek(-2, os.SEEK_CUR)
    print(file.readline().decode())

,,38063935,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci38063935&format=geojson,0.01698,,39.0,",ci38063935,",0.66,ml,,ci,24.0,"9km NE of Aguanga, CA",0.1,7,",ci,",reviewed,1537228864470,"M 0.7 - 9km NE of Aguanga, CA",0,earthquake,",focal-mechanism,geoserve,nearby-cities,origin,phase-data,scitech-link,",-480.0,1537305830770,https://earthquake.usgs.gov/earthquakes/eventpage/ci38063935


In [87]:
headers = !head -n 1 data/earthquakes.csv
len(headers[0].split(','))

1

### Reading in the file

In [94]:
#df = pd.read_csv('data/earthquakes.csv')

df = pd.read_csv(
    'https://github.com/stefmolin/'
    'Hands-On-Data-Analysis-with-Pandas-2nd-edition'
    '/blob/master/ch_02/data/earthquakes.csv?raw=True'
)

In [95]:
headers = !head -n 1 df
len(headers[0].split(','))

1

### Writing a DataFrame Object to a CSV File

In [91]:
df.to_csv('output.csv', index=False)

### Writing a DataFrame Object to a Database

In [92]:
import sqlite3

with sqlite3.connect('data/quakes.db') as connection:
    pd.read_csv('data/tsunamis.csv').to_sql(
        'tsunamis', connection, index=False, if_exists='replace'
    )

### Creating a DataFrame Object by Querying a Database

In [93]:
import sqlite3

with sqlite3.connect('data/quakes.db') as connection:
    tsunamis = pd.read_sql('SELECT * FROM tsunamis', connection)

tsunamis.head()

,alert,type,title,place,magType,mag,time
0,None,earthquake,"M 5.0 - 165km NNW of Flying Fish Cove, Christm...","165km NNW of Flying Fish Cove, Christmas Island",mww,5.0,1539459504090
1,green,earthquake,"M 6.7 - 262km NW of Ozernovskiy, Russia","262km NW of Ozernovskiy, Russia",mww,6.7,1539429023560
2,green,earthquake,"M 5.6 - 128km SE of Kimbe, Papua New Guinea","128km SE of Kimbe, Papua New Guinea",mww,5.6,1539312723620
3,green,earthquake,"M 6.5 - 148km S of Severo-Kuril'sk, Russia","148km S of Severo-Kuril'sk, Russia",mww,6.5,1539213362130
4,green,earthquake,"M 6.2 - 94km SW of Kokopo, Papua New Guinea","94km SW of Kokopo, Papua New Guinea",mww,6.2,1539208835130


# Making Pandas DataFrames from API Requests

In [98]:
# USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

In [96]:
import datetime as dt
import pandas as pd
import requests

# ask for earthquake data for the last 30 days
yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

In [97]:
# request OK?
response.status_code

200

In [99]:
#  JSON payload: a dictionary
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

In [100]:
earthquake_json['metadata']

{'generated': 1649092224000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2022-03-04&endtime=2022-04-03',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.13.3',
 'count': 9357}

In [101]:
type(earthquake_json['features'])

list

In [102]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.1,
  'place': '31 km SE of Mina, Nevada',
  'time': 1648943933835,
  'updated': 1648944173098,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/nn00836601',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nn00836601&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 19,
  'net': 'nn',
  'code': '00836601',
  'ids': ',nn00836601,',
  'sources': ',nn,',
  'types': ',origin,phase-data,',
  'nst': 10,
  'dmin': 0.055,
  'rms': 0.2159,
  'gap': 129.97,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.1 - 31 km SE of Mina, Nevada'},
 'geometry': {'type': 'Point', 'coordinates': [-117.8945, 38.1605, 11.1]},
 'id': 'nn00836601'}

In [103]:
# use a list comprehension to isolate the properties section from each of the dictionaries in the features list

earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.10,"31 km SE of Mina, Nevada",1648943933835,1648944173098,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nn00836601,",",nn,",",origin,phase-data,",10.0,0.055,0.2159,129.97,ml,earthquake,"M 1.1 - 31 km SE of Mina, Nevada"
1,1.60,"7 km ENE of Fox, Alaska",1648943529438,1648944127867,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak02248jg2bg,",",ak,",",origin,phase-data,",NaN,NaN,0.7800,NaN,ml,earthquake,"M 1.6 - 7 km ENE of Fox, Alaska"
2,1.20,"31 km S of Mina, Nevada",1648943506149,1648943726039,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nn00836599,",",nn,",",origin,phase-data,",14.0,0.073,0.2626,149.26,ml,earthquake,"M 1.2 - 31 km S of Mina, Nevada"
3,2.58,"13 km SSW of Tallaboa, Puerto Rico",1648943304730,1648944986790,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",pr71343138,",",pr,",",origin,phase-data,",23.0,NaN,0.2400,222.00,md,earthquake,"M 2.6 - 13 km SSW of Tallaboa, Puerto Rico"
4,1.30,"38 km SE of Mina, Nevada",1648943171273,1648943395592,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nn00836596,",",nn,",",origin,phase-data,",14.0,0.104,0.2483,144.28,ml,earthquake,"M 1.3 - 38 km SE of Mina, Nevada"


In [104]:
# write data to .csv

# df.to_csv('earthquakes.csv', index=False)

## Inspecting a DataFrame object

In [107]:
import numpy as np
import pandas as pd
df = pd.read_csv('data/earthquakes.csv')

print("is empty?", df.empty)
print("Shape?", df.shape)
print("Columns?", df.columns)


is empty? False
Shape? (9332, 26)
Columns? Index(['alert', 'cdi', 'code', 'detail', 'dmin', 'felt', 'gap', 'ids', 'mag',
       'magType', 'mmi', 'net', 'nst', 'place', 'rms', 'sig', 'sources',
       'status', 'time', 'title', 'tsunami', 'type', 'types', 'tz', 'updated',
       'url'],
      dtype='object')


In [109]:
print("Types?", df.dtypes)

Types? alert       object
cdi        float64
code        object
detail      object
dmin       float64
felt       float64
gap        float64
ids         object
mag        float64
magType     object
mmi        float64
net         object
nst        float64
place       object
rms        float64
sig          int64
sources     object
status      object
time         int64
title       object
tsunami      int64
type        object
types       object
tz         float64
updated      int64
url         object
dtype: object


In [110]:
print("Info?", df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9332 entries, 0 to 9331
Data columns (total 26 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alert    59 non-null     object 
 1   cdi      329 non-null    float64
 2   code     9332 non-null   object 
 3   detail   9332 non-null   object 
 4   dmin     6139 non-null   float64
 5   felt     329 non-null    float64
 6   gap      6164 non-null   float64
 7   ids      9332 non-null   object 
 8   mag      9331 non-null   float64
 9   magType  9331 non-null   object 
 10  mmi      93 non-null     float64
 11  net      9332 non-null   object 
 12  nst      5364 non-null   float64
 13  place    9332 non-null   object 
 14  rms      9332 non-null   float64
 15  sig      9332 non-null   int64  
 16  sources  9332 non-null   object 
 17  status   9332 non-null   object 
 18  time     9332 non-null   int64  
 19  title    9332 non-null   object 
 20  tsunami  9332 non-null   int64  
 21  type     9332 

In [111]:
df.head()

,alert,cdi,code,detail,dmin,felt,gap,ids,mag,magType,...,sources,status,time,title,tsunami,type,types,tz,updated,url
0,NaN,NaN,37389218,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.008693,NaN,85.0,",ci37389218,",1.35,ml,...,",ci,",automatic,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475395144,https://earthquake.usgs.gov/earthquakes/eventp...
1,NaN,NaN,37389202,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.020030,NaN,79.0,",ci37389202,",1.29,ml,...,",ci,",automatic,1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475253925,https://earthquake.usgs.gov/earthquakes/eventp...
2,NaN,4.4,37389194,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.021370,28.0,21.0,",ci37389194,",3.42,ml,...,",ci,",automatic,1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,earthquake,",dyfi,focal-mechanism,geoserve,nearby-cities,o...",-480.0,1539536756176,https://earthquake.usgs.gov/earthquakes/eventp...
3,NaN,NaN,37389186,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.026180,NaN,39.0,",ci37389186,",0.44,ml,...,",ci,",automatic,1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475196167,https://earthquake.usgs.gov/earthquakes/eventp...
4,NaN,NaN,73096941,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.077990,NaN,192.0,",nc73096941,",2.16,md,...,",nc,",automatic,1539474716050,"M 2.2 - 10km NW of Avenal, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,scit...",-480.0,1539477547926,https://earthquake.usgs.gov/earthquakes/eventp...


In [112]:
df.tail(2)

,alert,cdi,code,detail,dmin,felt,gap,ids,mag,magType,...,sources,status,time,title,tsunami,type,types,tz,updated,url
9330,NaN,NaN,38063959,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.01865,NaN,61.0,",ci38063959,",1.10,ml,...,",ci,",reviewed,1537229545350,"M 1.1 - 9km NE of Aguanga, CA",0,earthquake,",focal-mechanism,geoserve,nearby-cities,origin...",-480.0,1537230211640,https://earthquake.usgs.gov/earthquakes/eventp...
9331,NaN,NaN,38063935,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.01698,NaN,39.0,",ci38063935,",0.66,ml,...,",ci,",reviewed,1537228864470,"M 0.7 - 9km NE of Aguanga, CA",0,earthquake,",focal-mechanism,geoserve,nearby-cities,origin...",-480.0,1537305830770,https://earthquake.usgs.gov/earthquakes/eventp...


In [113]:
df.describe()

,cdi,dmin,felt,gap,mag,mmi,nst,rms,sig,time,tsunami,tz,updated
count,329.000000,6139.000000,329.000000,6164.000000,9331.000000,93.000000,5364.000000,9332.000000,9332.000000,9.332000e+03,9332.000000,9331.000000,9.332000e+03
mean,2.754711,0.544925,12.310030,121.506588,1.497345,3.651398,19.053878,0.362122,56.899914,1.538284e+12,0.006537,-451.990140,1.538537e+12
std,1.010637,2.214305,48.954944,72.962363,1.203347,1.790523,15.492315,0.317784,91.872163,6.080306e+08,0.080589,231.752571,6.564135e+08
min,0.000000,0.000648,0.000000,12.000000,-1.260000,0.000000,0.000000,0.000000,0.000000,1.537229e+12,0.000000,-720.000000,1.537230e+12
25%,2.000000,0.020425,1.000000,66.142500,0.720000,2.680000,8.000000,0.119675,8.000000,1.537793e+12,0.000000,-540.000000,1.537996e+12
50%,2.700000,0.059050,2.000000,105.000000,1.300000,3.720000,15.000000,0.210000,26.000000,1.538245e+12,0.000000,-480.000000,1.538621e+12
75%,3.300000,0.177250,5.000000,159.000000,1.900000,4.570000,25.000000,0.590000,56.000000,1.538766e+12,0.000000,-480.000000,1.539110e+12
max,8.400000,53.737000,580.000000,355.910000,7.500000,9.120000,172.000000,1.910000,2015.000000,1.539475e+12,1.000000,720.000000,1.539537e+12


In [117]:
# Specifying the 5th and 95th percentile:
df.describe(percentiles=[0.05, 0.95])

,cdi,dmin,felt,gap,mag,mmi,nst,rms,sig,time,tsunami,tz,updated
count,329.000000,6139.000000,329.000000,6164.000000,9331.000000,93.000000,5364.000000,9332.000000,9332.000000,9.332000e+03,9332.000000,9331.000000,9.332000e+03
mean,2.754711,0.544925,12.310030,121.506588,1.497345,3.651398,19.053878,0.362122,56.899914,1.538284e+12,0.006537,-451.990140,1.538537e+12
std,1.010637,2.214305,48.954944,72.962363,1.203347,1.790523,15.492315,0.317784,91.872163,6.080306e+08,0.080589,231.752571,6.564135e+08
min,0.000000,0.000648,0.000000,12.000000,-1.260000,0.000000,0.000000,0.000000,0.000000,1.537229e+12,0.000000,-720.000000,1.537230e+12
5%,2.000000,0.005491,1.000000,35.000000,-0.040000,0.000000,4.000000,0.030000,0.000000,1.537344e+12,0.000000,-600.000000,1.537387e+12
50%,2.700000,0.059050,2.000000,105.000000,1.300000,3.720000,15.000000,0.210000,26.000000,1.538245e+12,0.000000,-480.000000,1.538621e+12
95%,4.300000,2.678900,40.200000,276.000000,4.400000,6.380000,49.000000,0.960000,298.000000,1.539319e+12,0.000000,-60.000000,1.539400e+12
max,8.400000,53.737000,580.000000,355.910000,7.500000,9.120000,172.000000,1.910000,2015.000000,1.539475e+12,1.000000,720.000000,1.539537e+12


In [114]:
# only gives us summary statistics for non-null values
# if we had 100 rows and half of our data was null, 
# then the average would be calculated as the sum of the 50 non-null rows divided by 50
df.describe(include=np.object)

,alert,code,detail,ids,magType,net,place,sources,status,title,type,types,url
count,59,9332,9332,9332,9331,9332,9332,9332,9332,9332,9332,9332,9332
unique,2,9332,9332,9332,10,14,5433,52,2,7807,5,42,9332
top,green,20267746,https://earthquake.usgs.gov/fdsnws/event/1/que...,",mb80311099,",ml,ak,"10km NE of Aguanga, CA",",ak,",reviewed,"M 0.4 - 10km NE of Aguanga, CA",earthquake,",geoserve,origin,phase-data,",https://earthquake.usgs.gov/earthquakes/eventp...
freq,58,1,1,1,6803,3166,306,2981,7797,55,9081,5301,1


In [115]:
#unique(): Returns the distinct values of the column.
df.alert.unique()

array([nan, 'green', 'red'], dtype=object)

In [116]:
df.alert.value_counts()

green    58
red       1
Name: alert, dtype: int64

## Grabbing subsets of the data

In [118]:
import pandas as pd

df = pd.read_csv('data/earthquakes.csv')

In [119]:
df.mag

# oder df['mag']

0       1.35
1       1.29
2       3.42
3       0.44
4       2.16
        ... 
9327    0.62
9328    1.00
9329    2.40
9330    1.10
9331    0.66
Name: mag, Length: 9332, dtype: float64

In [120]:
df[['mag', 'title']]

,mag,title
0,1.35,"M 1.4 - 9km NE of Aguanga, CA"
1,1.29,"M 1.3 - 9km NE of Aguanga, CA"
2,3.42,"M 3.4 - 8km NE of Aguanga, CA"
3,0.44,"M 0.4 - 9km NE of Aguanga, CA"
4,2.16,"M 2.2 - 10km NW of Avenal, CA"
...,...,...
9327,0.62,"M 0.6 - 9km ENE of Mammoth Lakes, CA"
9328,1.00,"M 1.0 - 3km W of Julian, CA"
9329,2.40,"M 2.4 - 35km NNE of Hatillo, Puerto Rico"
9330,1.10,"M 1.1 - 9km NE of Aguanga, CA"


In [121]:
# Selecting columns using list comprehensions and string operations
# String methods:  https://docs.python.org/3/library/stdtypes.html#string-methods.

df[
    ['title', 'time']
    + [col for col in df.columns if col.startswith('mag')]
]

,title,time,mag,magType
0,"M 1.4 - 9km NE of Aguanga, CA",1539475168010,1.35,ml
1,"M 1.3 - 9km NE of Aguanga, CA",1539475129610,1.29,ml
2,"M 3.4 - 8km NE of Aguanga, CA",1539475062610,3.42,ml
3,"M 0.4 - 9km NE of Aguanga, CA",1539474978070,0.44,ml
4,"M 2.2 - 10km NW of Avenal, CA",1539474716050,2.16,md
...,...,...,...,...
9327,"M 0.6 - 9km ENE of Mammoth Lakes, CA",1537230228060,0.62,md
9328,"M 1.0 - 3km W of Julian, CA",1537230135130,1.00,ml
9329,"M 2.4 - 35km NNE of Hatillo, Puerto Rico",1537229908180,2.40,md
9330,"M 1.1 - 9km NE of Aguanga, CA",1537229545350,1.10,ml


In [123]:
# Slicing

#df[100:103]
df[['title', 'time']][100:103]

,title,time
100,"M 1.2 - 25km NW of Ester, Alaska",1539435449480
101,"M 0.6 - 8km ESE of Mammoth Lakes, CA",1539435391320
102,"M 1.3 - 8km ENE of Aguanga, CA",1539435293090


In [124]:
# use loc to select specific rows and columns without chaining. 
# If we use row numbers with loc, they are now inclusive of the end index

df.loc[10:15, ['title', 'mag']]

,title,mag
10,"M 0.5 - 10km NE of Aguanga, CA",0.50
11,"M 2.8 - 53km SE of Punta Cana, Dominican Republic",2.77
12,"M 0.5 - 9km NE of Aguanga, CA",0.50
13,"M 4.5 - 120km SSW of Banda Aceh, Indonesia",4.50
14,"M 2.1 - 14km NW of Parkfield, CA",2.13
15,"M 2.0 - 156km WNW of Haines Junction, Canada",2.00


In [125]:
# Exclusive of the endpoint just as Python slicing

df.iloc[10:15, [19, 8]]

,title,mag
10,"M 0.5 - 10km NE of Aguanga, CA",0.50
11,"M 2.8 - 53km SE of Punta Cana, Dominican Republic",2.77
12,"M 0.5 - 9km NE of Aguanga, CA",0.50
13,"M 4.5 - 120km SSW of Banda Aceh, Indonesia",4.50
14,"M 2.1 - 14km NW of Parkfield, CA",2.13


In [126]:
# use slicing syntax with iloc for both rows and columns

df.iloc[10:15, 6:10]

,gap,ids,mag,magType
10,57.0,",ci37389162,",0.50,ml
11,186.0,",pr2018286010,",2.77,md
12,76.0,",ci37389146,",0.50,ml
13,157.0,",us1000hbti,",4.50,mb
14,71.0,",nc73096921,",2.13,md


In [127]:
df.at[10, 'mag']

0.5

### Filtering

In [130]:
df.mag > 2

0       False
1       False
2        True
3       False
4        True
        ...  
9327    False
9328    False
9329     True
9330    False
9331    False
Name: mag, Length: 9332, dtype: bool

In [131]:
df.loc[
    df.mag >= 7.0,
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

,alert,mag,magType,title,tsunami,type
837,green,7.0,mww,"M 7.0 - 117km E of Kimbe, Papua New Guinea",1,earthquake
5263,red,7.5,mww,"M 7.5 - 78km N of Palu, Indonesia",1,earthquake


In [132]:
df.loc[
    (df.tsunami == 1) & (df.alert == 'red'),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

,alert,mag,magType,title,tsunami,type
5263,red,7.5,mww,"M 7.5 - 78km N of Palu, Indonesia",1,earthquake


In [133]:
df.loc[
    (df.place.str.contains('Alaska')) & (df.alert.notnull()),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

,alert,mag,magType,title,tsunami,type
1015,green,5.0,ml,"M 5.0 - 61km SSW of Chignik Lake, Alaska",1,earthquake
1273,green,4.0,ml,"M 4.0 - 71km SW of Kaktovik, Alaska",1,earthquake
1795,green,4.0,ml,"M 4.0 - 60km WNW of Valdez, Alaska",1,earthquake
2752,green,4.0,ml,"M 4.0 - 67km SSW of Kaktovik, Alaska",1,earthquake
3260,green,3.9,ml,"M 3.9 - 44km N of North Nenana, Alaska",0,earthquake
4101,green,4.2,ml,"M 4.2 - 131km NNW of Arctic Village, Alaska",0,earthquake
6897,green,3.8,ml,"M 3.8 - 80km SSW of Kaktovik, Alaska",0,earthquake
8524,green,3.8,ml,"M 3.8 - 69km SSW of Kaktovik, Alaska",0,earthquake
9133,green,5.1,ml,"M 5.1 - 64km SSW of Kaktovik, Alaska",1,earthquake


#### Between, IsIn and Like

In [135]:
df.loc[
    df.mag.between(6.5, 7.5),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

,alert,mag,magType,title,tsunami,type
118,green,6.7,mww,"M 6.7 - 262km NW of Ozernovskiy, Russia",1,earthquake
799,green,6.5,mww,"M 6.5 - 148km S of Severo-Kuril'sk, Russia",1,earthquake
837,green,7.0,mww,"M 7.0 - 117km E of Kimbe, Papua New Guinea",1,earthquake
4363,green,6.7,mww,"M 6.7 - 263km NNE of Ndoi Island, Fiji",1,earthquake
5263,red,7.5,mww,"M 7.5 - 78km N of Palu, Indonesia",1,earthquake


In [136]:
df.loc[
    df.magType.isin(['mw', 'mwb']),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

,alert,mag,magType,title,tsunami,type
995,NaN,3.35,mw,"M 3.4 - 9km WNW of Cobb, CA",0,earthquake
1465,green,3.83,mw,"M 3.8 - 109km WNW of Trinidad, CA",0,earthquake
2414,green,3.83,mw,"M 3.8 - 5km SW of Tres Pinos, CA",1,earthquake
4988,green,4.41,mw,"M 4.4 - 1km SE of Delta, B.C., MX",1,earthquake
6307,green,5.80,mwb,"M 5.8 - 297km NNE of Ndoi Island, Fiji",0,earthquake
8257,green,5.70,mwb,"M 5.7 - 175km SSE of Lambasa, Fiji",0,earthquake


In [138]:
df.filter(like='mag').head()

,mag,magType
0,1.35,ml
1,1.29,ml
2,3.42,ml
3,0.44,ml
4,2.16,md


In [140]:
df.set_index('place').filter(like='Japan', axis=0).filter(items=['mag', 'magType', 'title']).head()

,mag,magType,title
place,,,
"160km NNW of Nago, Japan",4.6,mb,"M 4.6 - 160km NNW of Nago, Japan"
"7km ESE of Asahi, Japan",5.2,mww,"M 5.2 - 7km ESE of Asahi, Japan"
"14km E of Tomakomai, Japan",4.5,mwr,"M 4.5 - 14km E of Tomakomai, Japan"
"139km WSW of Naze, Japan",4.7,mb,"M 4.7 - 139km WSW of Naze, Japan"
"53km ESE of Kamaishi, Japan",4.6,mb,"M 4.6 - 53km ESE of Kamaishi, Japan"


#### filtering with regular expressions

In [134]:
df.loc[
    (df.place.str.contains(r'CA|California$')) & (df.mag > 3.8),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

,alert,mag,magType,title,tsunami,type
1465,green,3.83,mw,"M 3.8 - 109km WNW of Trinidad, CA",0,earthquake
2414,green,3.83,mw,"M 3.8 - 5km SW of Tres Pinos, CA",1,earthquake


In [139]:
df.filter(regex=r'^t').head()

,time,title,tsunami,type,types,tz
0,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0
1,1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0
2,1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,earthquake,",dyfi,focal-mechanism,geoserve,nearby-cities,o...",-480.0
3,1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0
4,1539474716050,"M 2.2 - 10km NW of Avenal, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,scit...",-480.0


#### minimum and maximum values

In [137]:
df.loc[
    [df.mag.idxmin(), df.mag.idxmax()],
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

,alert,mag,magType,title,tsunami,type
2409,NaN,-1.26,ml,"M -1.3 - 41km ENE of Adak, Alaska",0,earthquake
5263,red,7.50,mww,"M 7.5 - 78km N of Palu, Indonesia",1,earthquake


## Adding and Removing Data

In [141]:
import pandas as pd

df = pd.read_csv(
    'data/earthquakes.csv', 
    usecols=['time', 'title', 'place', 'magType', 'mag', 'alert', 'tsunami']
)

In [143]:
# copy dataframe before making any changes
df_to_modify = df.copy()

### Creating new data

#### Adding new columns

In [144]:
# new columns can be achieved in the same fashion as variable assignment

df['source'] = 'USGS API' # broadcasting to set every row 
df.head()

,alert,mag,magType,place,time,title,tsunami,source
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,USGS API
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,USGS API
2,NaN,3.42,ml,"8km NE of Aguanga, CA",1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,USGS API
3,NaN,0.44,ml,"9km NE of Aguanga, CA",1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,USGS API
4,NaN,2.16,md,"10km NW of Avenal, CA",1539474716050,"M 2.2 - 10km NW of Avenal, CA",0,USGS API


In [145]:
# create a column that tells whether the earthquake's magnitude was negative

df['mag_negative'] = df.mag < 0
df.head()

,alert,mag,magType,place,time,title,tsunami,source,mag_negative
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,USGS API,False
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,USGS API,False
2,NaN,3.42,ml,"8km NE of Aguanga, CA",1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,USGS API,False
3,NaN,0.44,ml,"9km NE of Aguanga, CA",1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,USGS API,False
4,NaN,2.16,md,"10km NW of Avenal, CA",1539474716050,"M 2.2 - 10km NW of Avenal, CA",0,USGS API,False


#### data consistency issues

In [146]:
# multiple names for the same entity
# entity recognition problem

# look through and infer how to properly match up names
df.place.unique()

array(['9km NE of Aguanga, CA', '8km NE of Aguanga, CA',
       '10km NW of Avenal, CA', ..., '9km ENE of Mammoth Lakes, CA',
       '3km W of Julian, CA', '35km NNE of Hatillo, Puerto Rico'],
      dtype=object)

In [147]:
# use the replace() method to replace patterns in the place column

df['parsed_place'] = df.place.str.replace(
    r'.* of ', '', regex=True # remove anything saying <something> of <something>
).str.replace(
    'the ', '' # remove "the "
).str.replace(
    r'CA$', 'California', regex=True # fix California
).str.replace(
    r'NV$', 'Nevada', regex=True # fix Nevada
).str.replace(
    r'MX$', 'Mexico', regex=True # fix Mexico
).str.replace(
    r' region$', '', regex=True # chop off endings with " region"
).str.replace(
    'northern ', '' # remove "northern "
).str.replace(
    'Fiji Islands', 'Fiji' # line up the Fiji places
).str.replace(
    r'^.*, ', '', regex=True # remove anything else extraneous from the beginning
).str.strip() # remove any extra spaces

In [148]:
df.parsed_place.sort_values().unique()

array(['Afghanistan', 'Alaska', 'Argentina', 'Arizona', 'Arkansas',
       'Ascension Island', 'Australia', 'Azerbaijan', 'Balleny Islands',
       'Barbuda', 'Bolivia', 'British Virgin Islands', 'Burma',
       'California', 'Canada', 'Carlsberg Ridge',
       'Central East Pacific Rise', 'Central Mid-Atlantic Ridge', 'Chile',
       'China', 'Christmas Island', 'Colombia', 'Colorado', 'Costa Rica',
       'Dominican Republic', 'East Timor', 'Ecuador', 'El Salvador',
       'Fiji', 'Greece', 'Greenland', 'Guam', 'Guatemala', 'Haiti',
       'Hawaii', 'Honduras', 'Idaho', 'Illinois', 'India',
       'Indian Ocean Triple Junction', 'Indonesia', 'Iran', 'Iraq',
       'Italy', 'Jamaica', 'Japan', 'Kansas', 'Kentucky',
       'Kermadec Islands', 'Kuril Islands', 'Kyrgyzstan', 'Martinique',
       'Mauritius', 'Mayotte', 'Mexico', 'Mid-Indian Ridge', 'Missouri',
       'Montana', 'Nevada', 'New Caledonia', 'New Hampshire',
       'New Mexico', 'New Zealand', 'Nicaragua', 'North Carolina',


In [150]:
# more information  
# https://www.kdnuggets.com/2018/12/introduction-named-entity-recognition.html

#### make many new columns at once

In [151]:
# assign() method, the arguments are the names of the columns 
# want to create (or overwrite), and the values are the data for the columns
# sample() to randomly select five rows

df.assign(
    in_ca=df.parsed_place.str.endswith('California'),
    in_alaska=df.parsed_place.str.endswith('Alaska')
).sample(5, random_state=0)

# assign() doesn't change our original dataframe
# instead, it returns a new DataFrame object with these columns added

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place,in_ca,in_alaska
7207,NaN,4.80,mwr,"73km SSW of Masachapa, Nicaragua",1537749595210,"M 4.8 - 73km SSW of Masachapa, Nicaragua",0,USGS API,False,Nicaragua,False,False
4755,NaN,1.09,ml,"28km NNW of Packwood, Washington",1538227540460,"M 1.1 - 28km NNW of Packwood, Washington",0,USGS API,False,Washington,False,False
4595,NaN,1.80,ml,"77km SSW of Kaktovik, Alaska",1538259609862,"M 1.8 - 77km SSW of Kaktovik, Alaska",0,USGS API,False,Alaska,False,True
3566,NaN,1.50,ml,"102km NW of Arctic Village, Alaska",1538464751822,"M 1.5 - 102km NW of Arctic Village, Alaska",0,USGS API,False,Alaska,False,True
2182,NaN,0.90,ml,"26km ENE of Pine Valley, CA",1538801713880,"M 0.9 - 26km ENE of Pine Valley, CA",0,USGS API,False,California,True,False


In [152]:
#also accepts lambda functions

df.assign(
    in_ca=df.parsed_place == 'California',
    in_alaska=df.parsed_place == 'Alaska',
    neither=lambda x: ~x.in_ca & ~x.in_alaska
).sample(5, random_state=0)

# ~ is the bitwise negation operator
# allows us to create a column with the result of NOT in_ca AND NOT in_alaska per row

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place,in_ca,in_alaska,neither
7207,NaN,4.80,mwr,"73km SSW of Masachapa, Nicaragua",1537749595210,"M 4.8 - 73km SSW of Masachapa, Nicaragua",0,USGS API,False,Nicaragua,False,False,True
4755,NaN,1.09,ml,"28km NNW of Packwood, Washington",1538227540460,"M 1.1 - 28km NNW of Packwood, Washington",0,USGS API,False,Washington,False,False,True
4595,NaN,1.80,ml,"77km SSW of Kaktovik, Alaska",1538259609862,"M 1.8 - 77km SSW of Kaktovik, Alaska",0,USGS API,False,Alaska,False,True,False
3566,NaN,1.50,ml,"102km NW of Arctic Village, Alaska",1538464751822,"M 1.5 - 102km NW of Arctic Village, Alaska",0,USGS API,False,Alaska,False,True,False
2182,NaN,0.90,ml,"26km ENE of Pine Valley, CA",1538801713880,"M 0.9 - 26km ENE of Pine Valley, CA",0,USGS API,False,California,True,False,False


In [153]:
tsunami = df[df.tsunami == 1]
no_tsunami = df[df.tsunami == 0]
tsunami.shape, no_tsunami.shape

((61, 10), (9271, 10))

In [154]:
pd.concat([tsunami, no_tsunami]).shape

(9332, 10)

In [155]:
tsunami.append(no_tsunami).shape

(9332, 10)

####  Concatenating columns with matching indices

In [156]:
additional_columns = pd.read_csv(
    'data/earthquakes.csv', usecols=['tz', 'felt', 'ids']
)
pd.concat([df.head(2), additional_columns.head(2)], axis=1)

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place,felt,ids,tz
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,USGS API,False,California,NaN,",ci37389218,",-480.0
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,USGS API,False,California,NaN,",ci37389202,",-480.0


####  Concatenating columns with mismatching indices

In [157]:
additional_columns = pd.read_csv(
    'data/earthquakes.csv', usecols=['tz', 'felt', 'ids', 'time'], index_col='time'
)
pd.concat([df.head(2), additional_columns.head(2)], axis=1)

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place,felt,ids,tz
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1.539475e+12,"M 1.4 - 9km NE of Aguanga, CA",0.0,USGS API,False,California,NaN,NaN,NaN
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1.539475e+12,"M 1.3 - 9km NE of Aguanga, CA",0.0,USGS API,False,California,NaN,NaN,NaN
1539475129610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,",ci37389202,",-480.0
1539475168010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,",ci37389218,",-480.0


#### Appending rows and keeping only shared columns

In [158]:
pd.concat(
    [tsunami.head(2), no_tsunami.head(2).assign(type='earthquake')], join='inner'
)

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place
36,NaN,5.00,mww,"165km NNW of Flying Fish Cove, Christmas Island",1539459504090,"M 5.0 - 165km NNW of Flying Fish Cove, Christm...",1,USGS API,False,Christmas Island
118,green,6.70,mww,"262km NW of Ozernovskiy, Russia",1539429023560,"M 6.7 - 262km NW of Ozernovskiy, Russia",1,USGS API,False,Russia
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,USGS API,False,California
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,USGS API,False,California


#### Appending rows and resetting the index

In [159]:
pd.concat(
    [tsunami.head(2), no_tsunami.head(2).assign(type='earthquake')], join='inner', ignore_index=True
)

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place
0,NaN,5.00,mww,"165km NNW of Flying Fish Cove, Christmas Island",1539459504090,"M 5.0 - 165km NNW of Flying Fish Cove, Christm...",1,USGS API,False,Christmas Island
1,green,6.70,mww,"262km NW of Ozernovskiy, Russia",1539429023560,"M 6.7 - 262km NW of Ozernovskiy, Russia",1,USGS API,False,Russia
2,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,USGS API,False,California
3,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,USGS API,False,California


### Deleting Data

In [161]:
# catch KeyError if column does not exist
# alt.: df.pop('column_name') -> will leave a Boolean mask instead of column

try:
    del df['source']
except KeyError:
    # handle the error here
    pass

In [162]:
# index starts at 2 because dropped 0 and 1
df.drop([0, 1]).head(2)

,alert,mag,magType,place,time,title,tsunami,mag_negative,parsed_place
2,NaN,3.42,ml,"8km NE of Aguanga, CA",1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,False,California
3,NaN,0.44,ml,"9km NE of Aguanga, CA",1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,False,California


In [163]:
# This drops all the columns that aren't in the list we wanted to keep

cols_to_drop = [
    col for col in df.columns
    if col not in ['alert', 'mag', 'title', 'time', 'tsunami']
]
df.drop(columns=cols_to_drop).head()

,alert,mag,time,title,tsunami
0,NaN,1.35,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0
1,NaN,1.29,1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0
2,NaN,3.42,1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0
3,NaN,0.44,1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0
4,NaN,2.16,1539474716050,"M 2.2 - 10km NW of Avenal, CA",0


In [164]:
# with axis:

df.drop(columns=cols_to_drop).equals(
    df.drop(cols_to_drop, axis=1)
)

True

In [165]:
# By default, drop() will return a new DataFrame object; 
# however, if we really want to remove the data from our 
# original dataframe, we can pass in inplace=True

df.drop(columns=cols_to_drop, inplace=True)
df.head()

,alert,mag,time,title,tsunami
0,NaN,1.35,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0
1,NaN,1.29,1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0
2,NaN,3.42,1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0
3,NaN,0.44,1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0
4,NaN,2.16,1539474716050,"M 2.2 - 10km NW of Avenal, CA",0
